In [ ]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

# ERA5 Dataset

In [ ]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
import glob
import torch
import era5_data_proc
import os

RAW_ERA5_PATH = "/vol/bitbucket/bet20/dataset/era5/global_full"

## What does coarsen mean

In [ ]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/2022*.nc')
nc_files.sort()

data = xr.open_dataset(nc_files[0])
data

<xarray.Dataset> Size: 49GB
Dimensions:    (longitude: 1440, latitude: 721, level: 8, time: 124)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
  * time       (time) datetime64[ns] 992B 2022-01-01 ... 2022-01-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 8GB ...
    q          (time, level, latitude, longitude) float64 8GB ...
    t          (time, level, latitude, longitude) float64 8GB ...
    u          (time, level, latitude, longitude) float64 8GB ...
    v          (time, level, latitude, longitude) float64 8GB ...
    w          (time, level, latitude, longitude) float64 8GB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:
era5_uk_big = era5_data_proc.uk_big_subset(data)

In [ ]:
era5_uk_big

<xarray.Dataset> Size: 312MB
Dimensions:    (time: 124, level: 8, latitude: 81, longitude: 81)
Coordinates:
  * longitude  (longitude) float32 324B 347.0 347.2 347.5 347.8 ... 6.5 6.75 7.0
  * latitude   (latitude) float32 324B 65.0 64.75 64.5 64.25 ... 45.5 45.25 45.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
  * time       (time) datetime64[ns] 992B 2022-01-01 ... 2022-01-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 52MB 1.932e+05 ... ...
    q          (time, level, latitude, longitude) float64 52MB 3.179e-06 ... ...
    t          (time, level, latitude, longitude) float64 52MB 203.2 ... 280.2
    u          (time, level, latitude, longitude) float64 52MB 31.79 ... 2.22
    v          (time, level, latitude, longitude) float64 52MB 14.41 ... -0.4756
    w          (time, level, latitude, longitude) float64 52MB 0.02355 ... 0.632
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:
era5_uk_small = era5_data_proc.uk_small_subset(data)

In [ ]:
era5_uk_small

<xarray.Dataset> Size: 88MB
Dimensions:    (time: 124, level: 8, latitude: 46, longitude: 40)
Coordinates:
  * longitude  (longitude) float32 160B 352.0 352.2 352.5 ... 1.25 1.5 1.75
  * latitude   (latitude) float32 184B 61.0 60.75 60.5 ... 50.25 50.0 49.75
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
  * time       (time) datetime64[ns] 992B 2022-01-01 ... 2022-01-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 15MB 1.955e+05 ... ...
    q          (time, level, latitude, longitude) float64 15MB 3.179e-06 ... ...
    t          (time, level, latitude, longitude) float64 15MB 205.9 ... 277.2
    u          (time, level, latitude, longitude) float64 15MB 32.52 ... 2.99
    v          (time, level, latitude, longitude) float64 15MB 16.25 ... -7.766
    w          (time, level, latitude, longitude) float64 15MB -0.002549 ... ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:
sample = data.sel(time=data.time[0])
# sample
sample.coarsen(longitude=8, latitude=8, boundary="pad").mean()

<xarray.Dataset> Size: 6MB
Dimensions:    (level: 8, latitude: 91, longitude: 180)
Coordinates:
  * longitude  (longitude) float32 720B 0.875 2.875 4.875 ... 354.9 356.9 358.9
  * latitude   (latitude) float32 364B 89.12 87.12 85.12 ... -86.88 -88.88 -90.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
    time       datetime64[ns] 8B 2022-01-01
Data variables:
    z          (level, latitude, longitude) float64 1MB 1.88e+05 ... -667.6
    q          (level, latitude, longitude) float64 1MB 3.179e-06 ... 0.0007199
    t          (level, latitude, longitude) float64 1MB 194.4 194.4 ... 268.7
    u          (level, latitude, longitude) float64 1MB -13.91 ... -0.0004028
    v          (level, latitude, longitude) float64 1MB 4.011 4.469 ... 0.001661
    w          (level, latitude, longitude) float64 1MB 0.004755 ... -0.06573
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:
data